In [8]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [23]:
start_date = '20221226'
end_date = '20230219'

'20220801' to '20220828' 
'20220829' to '20221030'
'20221031' to '20221225'
'20221226' to '20230219'

In [24]:
query = f"""

with base as (
    select 
        date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
        date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_week, 
        city_name, order_type, order_id, customer_id,captain_id,
        distance_final_distance,
        service_obj_service_name service,
        ride_distance*0.8 ride_distance_threshold,
        spd_fraud_flag,ride_time,ride_distance,
        amount,amount_breakup_base_fare_total,actual_price
        
    from orders.order_logs_snapshot
    where order_status = 'dropped' --and event_type = 'dropped'
        and yyyymmdd BETWEEN '{start_date}' and '{end_date}'
        and service_obj_service_name = 'Link'
),

segment as (
select * 
from 
(
select 
    date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
    date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_wk, 
    captain_id, 
    case 
    when poc_segment like 'HP_D%' then '3. HP Daily'
    when poc_segment like 'HP_I%' then '4. HP Intra/Inter'

    when poc_segment like 'MP_D%' then '5. MP Daily'
    when poc_segment like 'MP_I%' then '6. MP Intra/Inter'

    when poc_segment like 'LP_D%' then '7. LP Daily'
    when poc_segment like 'LP_I%' then '8. LP Intra/Inter'
    
    when poc_segment like 'UHP_D%' then '1. UHP Daily'
    when poc_segment like 'UHP_I%' then '2. UHP Intra/Inter'
    
    end segment,
    row_number() over(partition by date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d'), captain_id order by yyyymmdd desc) seq_no
from datasets.captain_metrics_segments
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and poc_segment is not null 
    -- and day_of_week(cast (date_parse(yyyymmdd, '%Y%m%d') as date)) = 7
    and captain_id in (select distinct captain_id from base)
 ) where seq_no = 1
),

total_partial_offline_ride as (

select 
    order_week,
    service,
    count(distinct order_id) total_orders,
    count(distinct case when distance_final_distance <= ride_distance_threshold then order_id end) as total_partial_orders,
    count(distinct case when ride_distance <= 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as short_partial_orders,
    count(distinct case when ride_distance > 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as long_partial_orders
from base
group by 1,2
),


partial_offline_ride as (
select 
    order_week,
    service,
    coalesce(segment,'0. New') segment,
    count(distinct base.customer_id) uniq_customer,
    count(distinct base.captain_id) uniq_captain,
    count(distinct order_id) as total_orders,
    count(distinct case when distance_final_distance <= ride_distance_threshold then order_id end) as total_partial_orders,
    count(distinct case when distance_final_distance <= ride_distance_threshold then base.customer_id end) partial_uniq_customer,
    count(distinct case when distance_final_distance <= ride_distance_threshold then base.captain_id end) partial_uniq_captain,
    count(distinct case when ride_distance <= 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as short_partial_orders,
    count(distinct case when ride_distance > 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as long_partial_orders,
    sum( case when distance_final_distance <= ride_distance_threshold then amount_breakup_base_fare_total end) as actual_fare,
    sum( case when distance_final_distance <= ride_distance_threshold then actual_price end) as estimated_fare,
    sum( case when distance_final_distance <= ride_distance_threshold then 0.2*actual_price end) as expected_commission
from base
left join segment s on order_week = order_wk and s.captain_id = base.captain_id
group by 1,2,3
)

select 
    a.order_week,
    -- a.service,
    a.segment,
    a.total_orders,
    a.total_partial_orders,
    try(a.total_partial_orders*100.00/b.total_partial_orders) "partial_orders_%",
    a.expected_commission,
    a.short_partial_orders,
    a.long_partial_orders,
    -- try(a.short_partial_orders*100.00/b.total_partial_orders) short_partial_distr,
    -- try(a.long_partial_orders*100.00/b.total_partial_orders) long_partial_distr,
    try(a.short_partial_orders*100.00/b.short_partial_orders) "short_partial_%",
    try(a.long_partial_orders*100.00/b.long_partial_orders) "long_partial_%" ,
    a.actual_fare,
    a.estimated_fare,
    a.uniq_captain,
    a.partial_uniq_captain,
    a.uniq_customer,
    a.partial_uniq_customer
    
from partial_offline_ride a
left join total_partial_offline_ride b on a.order_week = b.order_week and a.service = b.service

order by 1 desc,2

"""

In [25]:
df_code = pd.read_sql(query, conn)
df_code 

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2023-02-13,0. New,8387,1171,0.50,6777.4,600,571,0.57,0.44,15032.0,33887.0,3528,928,8263,1150
1,2023-02-13,1. UHP Daily,520739,25427,10.82,176617.8,12929,12498,12.22,9.67,309675.0,883089.0,7379,5846,413525,24447
2,2023-02-13,2. UHP Intra/Inter,17987,1128,0.48,7421.6,747,381,0.71,0.29,14022.0,37108.0,851,401,17437,978
3,2023-02-13,3. HP Daily,930076,47997,20.42,338491.2,21282,26715,20.12,20.66,573957.0,1692456.0,23008,14760,697971,46065
4,2023-02-13,4. HP Intra/Inter,71458,3133,1.33,22525.6,1538,1595,1.45,1.23,37040.0,112628.0,4833,1790,68835,3108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,46897,1987,0.99,13307.0,880,1107,1.06,0.95,23050.0,66535.0,3491,1188,45585,1977
68,2022-12-26,5. MP Daily,1491908,81049,40.55,522749.4,33121,47928,39.86,41.04,960892.0,2613747.0,83161,35352,1012185,77243
69,2022-12-26,6. MP Intra/Inter,171584,8198,4.10,53516.4,3857,4341,4.64,3.72,96979.0,267582.0,25542,5524,160086,8076
70,2022-12-26,7. LP Daily,544524,39828,19.93,252479.2,15468,24360,18.62,20.86,455996.0,1262396.0,70044,19884,436718,38691


In [12]:
df1 = df_code
df1.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-08-22,0. New,512641,29743,16.93,185342.6,16014,13729,20.29,14.18,346838.5,926713.0,41488,15036,272234,27081
1,2022-08-22,1. UHP Daily,371052,14005,7.97,91423.8,6632,7373,8.40,7.62,176463.5,457119.0,5013,3801,288999,13548
2,2022-08-22,2. UHP Intra/Inter,11844,333,0.19,2212.0,208,125,0.26,0.13,4074.5,11060.0,536,206,11149,330
3,2022-08-22,3. HP Daily,771005,32258,18.36,218350.0,13825,18433,17.51,19.04,399721.0,1091750.0,18583,11624,557639,31037
4,2022-08-22,4. HP Intra/Inter,57551,2078,1.18,14483.4,969,1109,1.23,1.15,25069.0,72417.0,3978,1335,55163,2058


In [18]:
df2 = df_code
df2.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-10-24,0. New,945,626,0.43,2877.0,344,282,0.58,0.33,7464.0,14385.0,741,516,876,600
1,2022-10-24,1. UHP Daily,296792,11629,7.98,75024.2,5483,6146,9.19,7.14,146994.0,375121.0,5497,3586,250096,11344
2,2022-10-24,2. UHP Intra/Inter,10359,360,0.25,2455.2,186,174,0.31,0.20,4691.0,12276.0,583,220,9807,357
3,2022-10-24,3. HP Daily,609433,28749,19.73,184152.8,11809,16940,19.78,19.69,347094.5,920764.0,18470,10172,475724,27815
4,2022-10-24,4. HP Intra/Inter,42675,1933,1.33,12962.4,797,1136,1.34,1.32,21076.5,64812.0,3464,1046,41272,1684


In [22]:
df3 = df_code
df3.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-12-19,0. New,1626,917,0.44,4232.0,499,418,0.57,0.35,11227.5,21160.0,1192,763,1546,881
1,2022-12-19,1. UHP Daily,470625,19824,9.51,127155.4,9345,10479,10.60,8.72,247053.5,635777.0,6753,5160,367169,19081
2,2022-12-19,2. UHP Intra/Inter,12217,554,0.27,3524.0,322,232,0.37,0.19,6132.5,17620.0,593,267,11657,515
3,2022-12-19,3. HP Daily,962465,44193,21.21,282923.8,18046,26147,20.46,21.75,519855.0,1414619.0,23536,14601,703470,42387
4,2022-12-19,4. HP Intra/Inter,50578,2144,1.03,13834.0,1081,1063,1.23,0.88,24642.0,69170.0,3755,1288,49061,2115


In [26]:
df4 = df_code
df4.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2023-02-13,0. New,8387,1171,0.50,6777.4,600,571,0.57,0.44,15032.0,33887.0,3528,928,8263,1150
1,2023-02-13,1. UHP Daily,520739,25427,10.82,176617.8,12929,12498,12.22,9.67,309675.0,883089.0,7379,5846,413525,24447
2,2023-02-13,2. UHP Intra/Inter,17987,1128,0.48,7421.6,747,381,0.71,0.29,14022.0,37108.0,851,401,17437,978
3,2023-02-13,3. HP Daily,930076,47997,20.42,338491.2,21282,26715,20.12,20.66,573957.0,1692456.0,23008,14760,697971,46065
4,2023-02-13,4. HP Intra/Inter,71458,3133,1.33,22525.6,1538,1595,1.45,1.23,37040.0,112628.0,4833,1790,68835,3108


In [27]:
frames = [df1,df2,df3,df4]
results = pd.concat(frames)

In [28]:
results

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-08-22,0. New,512641,29743,16.93,185342.6,16014,13729,20.29,14.18,346838.5,926713.0,41488,15036,272234,27081
1,2022-08-22,1. UHP Daily,371052,14005,7.97,91423.8,6632,7373,8.40,7.62,176463.5,457119.0,5013,3801,288999,13548
2,2022-08-22,2. UHP Intra/Inter,11844,333,0.19,2212.0,208,125,0.26,0.13,4074.5,11060.0,536,206,11149,330
3,2022-08-22,3. HP Daily,771005,32258,18.36,218350.0,13825,18433,17.51,19.04,399721.0,1091750.0,18583,11624,557639,31037
4,2022-08-22,4. HP Intra/Inter,57551,2078,1.18,14483.4,969,1109,1.23,1.15,25069.0,72417.0,3978,1335,55163,2058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,46897,1987,0.99,13307.0,880,1107,1.06,0.95,23050.0,66535.0,3491,1188,45585,1977
68,2022-12-26,5. MP Daily,1491908,81049,40.55,522749.4,33121,47928,39.86,41.04,960892.0,2613747.0,83161,35352,1012185,77243
69,2022-12-26,6. MP Intra/Inter,171584,8198,4.10,53516.4,3857,4341,4.64,3.72,96979.0,267582.0,25542,5524,160086,8076
70,2022-12-26,7. LP Daily,544524,39828,19.93,252479.2,15468,24360,18.62,20.86,455996.0,1262396.0,70044,19884,436718,38691


In [29]:
results.to_csv("partial_link.csv", index = False)